In [1]:
import psycopg2
import numpy as np
from os import environ, chdir; chdir('../')
conn = psycopg2.connect(dbname='postgres', 
                        user='postgres', 
                        host=environ['REDHAT_POSTGRES_1_PORT_5432_TCP_ADDR'])
cur = conn.cursor()

In [2]:
from lib.helpers.database_helper import pull_actions, one_hot_and_outcome
from numpy.linalg import norm

In [3]:
action_ids = pull_actions(action_type='training')

In [4]:
len(action_ids)

1000

In [5]:
batch_features = []
batch_outcomes = []
n=100
for action in action_ids[n:n+10]:
    this_one_hot_vec, \
        this_outcome = one_hot_and_outcome(action)
    batch_features.append(np.append(this_one_hot_vec, np.ones(1)))
    batch_outcomes.append(this_outcome)
batch_features = np.array(batch_features)
batch_outcomes = np.array(batch_outcomes)
batch_features.shape, batch_outcomes.shape, batch_outcomes

((10, 7326), (10,), array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0]))

In [6]:
W = np.random.rand(2,7326)  # random weight matrix

In [7]:
scores = W.dot(batch_features.T)

In [8]:
scores.T, batch_outcomes

(array([[ 37.73721381,  53.59685328],
        [ 27.75938086,  32.14392221],
        [ 27.95305603,  31.97511924],
        [ 37.41869667,  53.90123136],
        [ 37.41869667,  53.90123136],
        [ 37.41869667,  53.90123136],
        [ 37.41869667,  53.90123136],
        [ 27.75938086,  32.14392221],
        [ 37.41869667,  53.90123136],
        [ 27.75938086,  32.14392221]]), array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0]))

In [9]:
def loss_function(scores, y_batch, delta=1.0, gamma=0.1):
    correct_score = scores.T[np.arange(len(scores.T)), y_batch]
    margins = np.maximum(0, scores - correct_score + delta).T
    margins[np.arange(len(margins)), y_batch] = 0
    return np.sum(margins, axis=1) + gamma*norm(W)

In [10]:
loss_function(scores,batch_outcomes,)

array([  7.00061402,  12.38515538,  12.02267723,   7.00061402,
         7.00061402,   7.00061402,   7.00061402,  12.38515538,
         7.00061402,  12.38515538])

In [13]:
bestloss = float("inf") # Python assigns the highest possible float value
for num in range(1000):
    W = np.random.randn(10, 7326) * 0.0001 # generate random parameters
    loss = loss_function(scores, batch_outcomes) # get the loss over the entire training set
    if loss < bestloss: # keep track of the best solution
        bestloss = loss
        bestW = W
    print('in attempt {} the loss was {}, best {}'.format(num, loss, bestloss))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()